In [1]:
import sys
sys.path.append("..")  # <-  This should point to the root directory of the project relative to this file

from scripts import database

In [2]:
example_query =  """
                SELECT * FROM flights
                    LIMIT 10;
                """

example = database.query(example_query)

In [3]:
example

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-10-01,G4,G4,G4,169,G4,301NV,169,11603,EUG,...,700.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
1,2018-10-01,G4,G4,G4,589,G4,233NV,589,10849,BZN,...,861.0,4.0,0.0,0.0,0.0,42.0,None,None,None,None
2,2018-10-01,G4,G4,G4,30,G4,330NV,30,12889,LAS,...,1205.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
3,2018-10-01,G4,G4,G4,1713,G4,228NV,1713,11996,GSP,...,620.0,0.0,0.0,0.0,0.0,111.0,None,None,None,None
4,2018-10-01,G4,G4,G4,1591,G4,257NV,1591,11721,FNT,...,1111.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
5,2018-10-01,G4,G4,G4,549,G4,232NV,549,14512,RFD,...,1373.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
6,2018-10-01,G4,G4,G4,1502,G4,221NV,1502,12917,LCK,...,539.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
7,2018-10-01,G4,G4,G4,2153,G4,883GA,2153,12265,IAG,...,996.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
8,2018-10-01,G4,G4,G4,1692,G4,221NV,1692,14082,PGD,...,890.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
9,2018-10-01,G4,G4,G4,1900,G4,224NV,1900,15624,VPS,...,600.0,21.0,0.0,4.0,0.0,0.0,None,None,None,None
